# Test utils

## Import Data

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import sys
import string
import re
import nltk

print(sys.version)
print(sys.executable)

# Specify the path to your XML file
xml_file_path = r'..\data\test_data\sample.xml'

# Define the namespacetree
namespace = {'ns': 'req_document.xsd'}

# Parse the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
c:\ProgramData\anaconda3\python.exe


### Read XML into Dic

In [2]:
import utils.parse_xml as ParseXML

# Define the namespacetree
namespace = {}

df = ParseXML.parse_xml_to_dataframe(xml_file_path, namespace)

display(df.head(10))

,tag,text,id,path
0,root,,,/root
1,item,,1,root/item
2,name,Item 1,,root/item/name
3,price,10.00,,root/item/price
4,item,,2,root/item
5,name,Item 2,,root/item/name
6,price,20.00,,root/item/price
